In [6]:
import os
import time
import requests
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service  # <-- needed
from selenium.webdriver.common.by import By
from tqdm import tqdm


In [3]:
ingredient_to_category = {
    # Nuts
    "peanuts": "Nuts",
    "cashew": "Nuts",
    "chestnuts": "Nuts",
    "pistachios": "Nuts",
    "almond": "Nuts",
    "hazelnut": "Nuts",
    "walnuts": "Nuts",
    "pecans": "Nuts",
    "brazil_nut": "Nuts",
    "pili_nut": "Nuts",
    
    # Spices
    "cumin": "Spices",
    "star_anise": "Spices",
    "nutmeg": "Spices",
    "cloves": "Spices",
    "ginger": "Spices",
    "allspice": "Spices",
    "chervil": "Spices",
    "mustard": "Spices",
    "cinnamon": "Spices",
    "saffron": "Spices",
    
    # Herbs
    "angelica": "Herbs",
    "garlic": "Herbs",
    "chives": "Herbs",
    "turnip": "Herbs",
    "dill": "Herbs",
    "mugwort": "Herbs",
    "chamomile": "Herbs",
    "coriander": "Herbs",
    "oregano": "Herbs",
    "mint": "Herbs",
    
    # Fruits
    "kiwi": "Fruits",
    "pineapple": "Fruits",
    "banana": "Fruits",
    "lemon": "Fruits",
    "mandarin_orange": "Fruits",
    "strawberry": "Fruits",
    "apple": "Fruits",
    "mango": "Fruits",
    "peach": "Fruits",
    "pear": "Fruits",
    
    # Vegetables
    "cauliflower": "Vegetables",
    "brussel_sprouts": "Vegetables",
    "broccoli": "Vegetables",
    "sweet_potato": "Vegetables",
    "asparagus": "Vegetables",
    "avocado": "Vegetables",
    "radish": "Vegetables",
    "tomato": "Vegetables",
    "potato": "Vegetables",
    "cabbage": "Vegetables",
}


In [7]:
# === Configuration ===
INGREDIENTS = list(ingredient_to_category.keys())
SAVE_DIR = "ingredient_images"
IMAGES_PER_INGREDIENT = 10  # Change to more if needed
CHROMEDRIVER_PATH = "chromedriver"  # Update if not in PATH

# === Set up Selenium headless Chrome ===
os.makedirs(SAVE_DIR, exist_ok=True)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
# Create driver using Service object
service = Service(CHROMEDRIVER_PATH)
driver = webdriver.Chrome(service=service, options=chrome_options)

# === Function to scrape one image ===
def scrape_images(query, max_images=1):
    driver.get(f"https://www.google.com/search?q={query}&tbm=isch")
    time.sleep(2)
    thumbnails = driver.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
    image_urls = []

    for img in thumbnails[:max_images]:
        try:
            img.click()
            time.sleep(1)
            actual_images = driver.find_elements(By.CSS_SELECTOR, "img.n3VNCb")
            for actual_img in actual_images:
                src = actual_img.get_attribute("src")
                if src and "http" in src:
                    image_urls.append(src)
                    break
        except Exception as e:
            print(f"Error retrieving image for {query}: {e}")
    return image_urls

# === Download images ===
for ingredient in tqdm(INGREDIENTS):
    urls = scrape_images(ingredient + " food")
    for i, url in enumerate(urls):
        try:
            response = requests.get(url, timeout=10)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            image.save(os.path.join(SAVE_DIR, f"{ingredient}_{i+1}.jpg"))
        except Exception as e:
            print(f"Failed to download {ingredient}: {e}")

driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
